In [ ]:
import dask, concurrent.futures, time, warnings, os, re, pickle
from osgeo import gdal
import os
import glob
import requests as r
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio
import rioxarray as riox
import time
import xarray as xr
from urllib.request import urlopen
from xml.etree.ElementTree import parse,fromstring
from pandas import to_datetime
from rasterio.crs import CRS
from datetime import datetime, timedelta
from netrc import netrc
from pyproj import Proj
from src.hls_funcs import fetch
from src.hls_funcs.masks import mask_hls, shp2mask, bolton_mask
from src.hls_funcs.indices import ndvi_func
from src.hls_funcs.smooth import smooth_xr, despike_ts_xr
import cartopy.crs as ccrs
from rasterio.plot import show
from src.hls_funcs.predict import pred_bm, pred_bm_se, pred_cov
import dask.diagnostics
import matplotlib.pyplot as plt
import gc


In [ ]:
#wkDIR = '/project/cper_neon_aop/hls_nrt/'
#outDIR = '/90daydata/cper_neon_aop/hls_nrt/' + prefix

wkDIR = os.getcwd()
outDIR_base = 'data/hls_nrt/'

cluster_loc = 'local'

In [ ]:
yr = 2022

#prefix = 'redtop' 
prefix = 'cper'

if prefix == 'cper': 
    aoi_f = os.path.join('data/ground/cper_pastures_2017_clip.shp')
    df_aoi = gpd.read_file(aoi_f)
    subunit_name_old = 'Past_Name_'
    subunit_name = 'Pasture'
elif prefix == 'redtop':
    from src.utils.convert import kmz_to_shp
    df_aoi = kmz_to_shp('data/ground/RedTop_Boundary.kmz', 'data/ground/')
    df_aoi = df_aoi.to_crs(epsg=32613)
    subunit_name_old = None
    subunit_name = None

outDIR = os.path.join(outDIR_base, prefix)

mod_bm = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
mod_cov = pickle.load(open('src/models/CPER_HLS_to_LPI_cover_pls_binned_model.pk', 'rb'))

In [ ]:
os.chdir(wkDIR)
if cluster_loc == 'local':
    print('   setting up Local cluster...')
    from dask.distributed import LocalCluster, Client
    import dask
    aws=False
    fetch.setup_env(aws=aws)
    smth_chunks = {'y': 30, 'x': 30, 'time':-1}
    cluster = LocalCluster(n_workers=8, threads_per_worker=2)
    client = Client(cluster)
    display(client)
elif cluster_loc == 'coiled':
    import coiled
    aws=True
    fetch.setup_env(aws=aws)
    smth_chunks = {'y': 10, 'x': 10, 'time':-1}
    s3_cred = fetch.setup_netrc(creds=['spkearney', '1mrChamu'], aws=aws)
    coiled.create_software_environment(
    name="hls_cog_coiled",
    conda="hls_cog_coiled_env.yaml")
    cluster = coiled.Cluster(
        name="hls_cog_coiled",
        software="kearney-sp/hls_cog_coiled",
        n_workers=5,
        worker_cpu=2,
        scheduler_cpu=2,
        backend_options={"region": "us-west-2"},
        environ=dict(GDAL_DISABLE_READDIR_ON_OPEN='FALSE', 
                   #AWS_NO_SIGN_REQUEST='YES',
                   GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
                   GDAL_SWATH_SIZE='200000000',
                   VSI_CURL_CACHE_SIZE='200000000',
                   CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                   GDAL_HTTP_UNSAFESSL='YES',
                   GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                   GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
                   AWS_REGION='us-west-2',
                   AWS_SECRET_ACCESS_KEY=s3_cred['secretAccessKey'],
                   AWS_ACCESS_KEY_ID=s3_cred['accessKeyId'],
                   AWS_SESSION_TOKEN=s3_cred['sessionToken'])
    )
    client = Client(cluster)
    display(client)
elif cluster_loc == 'hpc':
    from dask.distributed import LocalCluster, Client
    import dask_jobqueue as jq
    import dask
    from jupyter_server import serverapp
    
    # get the server address for porting
    try:
        jupServer = [x for x in serverapp.list_running_servers()][0]
    except IndexError:
        # manually copy/paste the server address
        jupServer = {'base_url': '/node/ceres19-compute-98-eth.scinet.local/17710/'}
    print('   setting up moderate cluster on HPC...')
    aws=False
    fetch.setup_env(aws=aws)
    smth_chunks = {'y': 10, 'x': 10, 'time':-1}
    dask.config.set({'distributed.dashboard.link': jupServer['base_url'] + 'proxy/{port}/status'})
    partition='short',#'short','debug', 'mem', 'mem-low',
    num_processes = 4
    num_threads_per_processes = 2
    mem = 2.5*num_processes*num_threads_per_processes
    n_cores_per_job = num_processes*num_threads_per_processes
    clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            #interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
    client=Client(clust)
    #Scale Cluster 
    num_jobs=20
    clust.scale(jobs=num_jobs)
    try:
        client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
    except dask.distributed.TimeoutError as e:
        print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
        #print(e)
        pass
    display(client)

if not os.path.exists(outDIR):
    os.mkdir(outDIR)

if subunit_name_old is not None:
    df_aoi = df_aoi.rename(columns={subunit_name_old: subunit_name})
if subunit_name is not None:
    df_aoi = df_aoi.dissolve(by=subunit_name).reset_index()

start_date = str(yr - 1) + "-11-01"
end_date = str(yr + 1) + "-03-01"

# set the date range for analysis
date_rng = pd.date_range(start=start_date, end=end_date)
date_rng = date_rng[date_rng <= datetime.today()]

In [ ]:
t00 = time.time()
nc_flist = list(filter(os.path.isfile, 
                    glob.glob(os.path.join(outDIR,
                              prefix + 
                              '_hls_tmp/' + 'hls_ds_' + str(yr) + '*.nc'))))
if len(nc_flist) > 0:
    nc_flist.sort(key=lambda x: os.path.getctime(x))
    nc_f = nc_flist[-1]
else:
    print('ERROR: No temp nc file found')

hls_ds = xr.open_dataset(nc_f)

In [ ]:
hls_ds

In [ ]:
t0 = time.time()
print('   applying secondary cloud mask...')
hls_ds = hls_ds.chunk({'time': -1, 'y': 100, 'x': 100})
t1 = time.time()
hls_bolton_mask = bolton_mask(hls_ds)
# mask out scenes where < 75% of the region is cloud-free
partial_mask = (hls_bolton_mask.count(dim=['y', 'x'])/np.product(hls_bolton_mask.shape[1:])) > 0.75
hls_ds = hls_ds.where(hls_bolton_mask == 0)
hls_ds = hls_ds.where(partial_mask).compute()
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

hls_ds = hls_ds.chunk({'time': 1, 'y': -1, 'x': -1})
print('   computing NDVI for available dates...')
t1 = time.time()
# create ndvi array
xr_ndvi = ndvi_func(hls_ds)#.compute()
xr_ndvi.name = 'NDVI'
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

print('   computing biomass for available dates...')
t1 = time.time()
# create biomass array
xr_bm = hls_ds.map_blocks(pred_bm, template=hls_ds['NIR1'],
                          kwargs=dict(model=mod_bm)).where(hls_ds['NIR1'].notnull())#.compute()
xr_bm.name = 'Biomass'
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

print('   computing biomass SE for available dates...')
t1 = time.time()
# create biomass array
xr_bm_se = hls_ds.map_blocks(pred_bm_se, template=hls_ds['NIR1'],
                          kwargs=dict(model=mod_bm)).where(hls_ds['NIR1'].notnull())#.compute()
xr_bm_se.name = 'Biomass_SE'
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

print('   computing cover for available dates...')
t1 = time.time()
# create cover array
xr_cov = pred_cov(hls_ds, model=mod_cov).where(hls_ds['NIR1'].notnull())#.compute()
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

print('   removing dates with > 75% cloud cover...')
t1 = time.time()

xr_ndvi = xr_ndvi.where(partial_mask, drop=True)
xr_bm = xr_bm.where(partial_mask, drop=True)
xr_bm_se = xr_bm_se.where(partial_mask, drop=True)
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

In [ ]:
print('   creating daily template for output...')
t1 = time.time()
# create an output Dataset template with all dates
dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_ndvi['time'].values],
                             'x': xr_ndvi.x,
                             'y': xr_ndvi.y})

In [ ]:
dat_out_nans = np.zeros((dat_out.dims['time'], 
                         dat_out.dims['y'], 
                         dat_out.dims['x'])) * np.nan

In [ ]:
dat_out = dat_out.assign(NDVI=(['time', 'y', 'x'],
                               dat_out_nans))

In [ ]:
dat_out_ds = xr.concat([dat_out['NDVI'], xr_ndvi], dim='time').to_dataset()
dat_out_ds = dat_out_ds.sortby('time')
dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

In [ ]:
dat_out_ds

In [ ]:
yr_mask = dat_out_ds['time'].astype(np.datetime64).dt.year == yr

if not os.path.exists(os.path.join(outDIR, 'hls_ndvi/')):
    os.mkdir(os.path.join(outDIR, 'hls_ndvi/'))
if not os.path.exists(os.path.join(outDIR, 'hls_biomass/')):
    os.mkdir(os.path.join(outDIR, 'hls_biomass/'))
if not os.path.exists(os.path.join(outDIR, 'hls_cover/')):
    os.mkdir(os.path.join(outDIR, 'hls_cover/'))

In [ ]:
dat_out_ds['NDVI_despike'] = despike_ts_xr(dat_out_ds['NDVI'].chunk(smth_chunks), 
                                           days_thresh=45,
                                           dat_thresh=0.07, 
                                           mask_outliers=False,
                                           iters=2,
                                           dims=['time'])
dat_out_ds['NDVI_smooth'] = smooth_xr(dat_out_ds['NDVI_despike'], 
                                      dims=['time'], 
                                      kwargs={'double': True, 'limit': 91})

dat_out_ds.sel(time=yr_mask)['NDVI_smooth'].astype('float32').rio.reproject("EPSG:3857").to_netcdf(
    os.path.join(outDIR, 'hls_ndvi/' + prefix + '_hls_ndvi_' + str(yr) + '.nc'))

dat_out_ds = dat_out_ds.drop_vars([k for k in dat_out_ds.data_vars.keys() if 'NDVI' in k])
gc.collect()

In [ ]:
dat_out = dat_out.rename({'NDVI': 'Biomass'})
dat_out_ds['Biomass'] = xr.concat([dat_out['Biomass'], xr_bm], dim='time')
dat_out_ds = dat_out_ds.sortby('time')

dat_out_ds['Biomass_despike'] = despike_ts_xr(dat_out_ds['Biomass'].chunk(smth_chunks),
                                              days_thresh=45,
                                              dat_thresh=150, 
                                              mask_outliers=True,
                                              z_thresh=5.0,
                                              iters=2,
                                              dims=['time'])
dat_out_ds['Biomass_smooth'] = smooth_xr(dat_out_ds['Biomass_despike'], 
                                         dims=['time'],
                                         kwargs={'double': True, 'limit': 91})

dat_out_ds.sel(time=yr_mask)['Biomass_smooth'].astype('float32').rio.reproject("EPSG:3857").to_netcdf(
    os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_' + str(yr) + '.nc'))

dat_out_ds = dat_out_ds.drop_vars([k for k in dat_out_ds.data_vars.keys() if 'Biomass' in k and 'SE' not in k])
gc.collect()

In [ ]:
dat_out = dat_out.rename({'Biomass': 'Biomass_SE'})
dat_out_ds['Biomass_SE'] = xr.concat([dat_out['Biomass_SE'], xr_bm_se], dim='time')

dat_out_ds['Biomass_SE_despike'] = despike_ts_xr(dat_out_ds['Biomass_SE'].chunk(smth_chunks),
                                                 days_thresh=45,
                                                 dat_thresh=dat_out_ds['Biomass_SE'].std('time').mean().values * 3.0, 
                                                 mask_outliers=True,
                                                 z_thresh=5.0,
                                                 iters=2,
                                                 dims=['time'])
dat_out_ds['Biomass_SE_smooth'] = smooth_xr(dat_out_ds['Biomass_SE_despike'], 
                                         dims=['time'],
                                         kwargs={'double': True, 'limit': 91})

dat_out_ds.sel(time=yr_mask)['Biomass_SE_smooth'].astype('float32').rio.reproject("EPSG:3857").to_netcdf(
    os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_se_' + str(yr) + '.nc'))

dat_out_ds = dat_out_ds.drop_vars([k for k in dat_out_ds.data_vars.keys() if 'Biomass_SE' in k])
gc.collect()

In [ ]:
c_prev = 'Biomass_SE'

for c in ['BARE', 'SD', 'GREEN', 'LITT']:
    print(c)
    dat_out = dat_out.rename({c_prev: c})
    dat_out_ds[c] = xr.concat([dat_out[c], xr_cov[c]], dim='time')
    
    dat_out_ds[c + '_despike'] = despike_ts_xr(dat_out_ds[c].chunk(smth_chunks),
                                               days_thresh=45,
                                               dat_thresh=0.3, 
                                               mask_outliers=True,
                                               z_thresh=5.0,
                                               iters=2,
                                               dims=['time'])
    dat_out_ds[c + '_smooth'] = smooth_xr(dat_out_ds[c + '_despike'], 
                                         dims=['time'],
                                         kwargs={'double': True, 'limit': 91})

    dat_out_ds.sel(time=yr_mask)[c + '_smooth'].astype('float32').rio.reproject("EPSG:3857").to_netcdf(
        os.path.join(outDIR, 'hls_cover/' + prefix + '_hls_' + c + '_' + str(yr) + '.nc'))
    
    dat_out_ds = dat_out_ds.drop_vars([k for k in dat_out_ds.data_vars.keys() if c in k])
    gc.collect()
    
    c_prev = c

In [ ]:
gc.collect()

/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/src/hls_funcs/masks.py:121: RuntimeWarning: divide by zero encountered in double_scalars
  shadow_val = swir2_diff / (swir2_ts[idx_post] - swir2_ts[idx_pre])
/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/src/hls_funcs/masks.py:121: RuntimeWarning: invalid value encountered in double_scalars
  shadow_val = swir2_diff / (swir2_ts[idx_post] - swir2_ts[idx_pre])
/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/src/hls_funcs/masks.py:103: RuntimeWarning: divide by zero encountered in double_scalars
  shadow_val = swir2_diff / (swir2_ts[idx_pre] - swir2_ts[idx_pre2])
/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/src/hls_funcs/masks.py:103: RuntimeWarning: divide by zero encountered in double_scalars
  shadow_val = swir2_diff / (swir2_ts[idx_pre] - swir2_ts[idx_pre2])
/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/src/hls_funcs/masks.py:121: RuntimeWarning: invalid value encountered in double_scalars
  shadow_val = swir2_diff / (swir2_ts[idx_post] - swir2_ts[idx_pre

In [ ]:
yr_mask = dat_out_ds['time'].astype(np.datetime64).dt.year == yr
out_profile = {'driver': 'GTiff',
               'dtype': 'float32',
               'nodata': -9999.,
               'width': len(dat_out_ds.x),
               'height': len(dat_out_ds.y),
               'count': len(dat_out_ds.sel(time=yr_mask).time),
               'crs': CRS.from_dict(init='epsg:32613'),
               'transform': rio.Affine(30.0, 0.0, np.min(dat_out_ds.x).data,
                                       0.0, -30.0, np.max(dat_out_ds.y).data),
               'tiled': False}

if not os.path.exists(os.path.join(outDIR, 'hls_ndvi/')):
    os.mkdir(os.path.join(outDIR, 'hls_ndvi/'))
if not os.path.exists(os.path.join(outDIR, 'hls_biomass/')):
    os.mkdir(os.path.join(outDIR, 'hls_biomass/'))
if not os.path.exists(os.path.join(outDIR, 'hls_cover/')):
    os.mkdir(os.path.join(outDIR, 'hls_cover/'))

In [ ]:
dat_out_ds['NDVI_despike'] = despike_ts_xr(dat_out_ds['NDVI'].chunk(smth_chunks), 
                                           days_thresh=45,
                                           dat_thresh=0.07, 
                                           mask_outliers=False,
                                           iters=2,
                                           dims=['time'])
dat_out_ds['NDVI_smooth'] = smooth_xr(dat_out_ds['NDVI_despike'], 
                                      dims=['time'], 
                                      kwargs={'double': True, 'limit': 91})

with rio.open(os.path.join(outDIR, 'hls_ndvi/' + prefix + '_hls_ndvi_' + str(yr) + '.tif'), 
              'w',
              **out_profile) as dst:
    dst.write(dat_out_ds.sel(time=yr_mask)['NDVI_smooth'].astype('float32'))

In [ ]:
dat_out_ds.time.dt.date

In [ ]:
dat_out = dat_out.rename({'NDVI': 'Biomass'})
dat_out_ds['Biomass'] = xr.concat([dat_out['Biomass'], xr_bm], dim='time')
dat_out_ds = dat_out_ds.sortby('time')

dat_out_ds['Biomass_despike'] = despike_ts_xr(dat_out_ds['Biomass'].chunk(smth_chunks),
                                              days_thresh=45,
                                              dat_thresh=150, 
                                              mask_outliers=True,
                                              z_thresh=5.0,
                                              iters=2,
                                              dims=['time'])
dat_out_ds['Biomass_smooth'] = smooth_xr(dat_out_ds['Biomass_despike'], 
                                         dims=['time'],
                                         kwargs={'double': True, 'limit': 91})

with rio.open(os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_' + str(yr) + '.tif'), 
              'w',
              **out_profile) as dst:
    dst.write(dat_out_ds.sel(time=yr_mask)['Biomass_smooth'].astype('float32'))

In [ ]:
dat_out = dat_out.rename({'Biomass': 'Biomass_SE'})
dat_out_ds['Biomass_SE'] = xr.concat([dat_out['Biomass_SE'], xr_bm], dim='time')

dat_out_ds['Biomass_SE_despike'] = despike_ts_xr(dat_out_ds['Biomass_SE'].chunk(smth_chunks),
                                                 days_thresh=45,
                                                 dat_thresh=dat_out_ds['Biomass_SE'].std('time').mean().values * 3.0, 
                                                 mask_outliers=True,
                                                 z_thresh=5.0,
                                                 iters=2,
                                                 dims=['time'])
dat_out_ds['Biomass_SE_smooth'] = smooth_xr(dat_out_ds['Biomass_SE_despike'], 
                                         dims=['time'],
                                         kwargs={'double': True, 'limit': 91})

with rio.open(os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_se_' + str(yr) + '.tif'), 
              'w',
              **out_profile) as dst:
    dst.write(dat_out_ds.sel(time=yr_mask)['Biomass_SE_smooth'].astype('float32'))

In [ ]:
c_prev = 'Biomass_SE'

for c in ['BARE', 'SD', 'GREEN', 'LITT']:
    print(c)
    dat_out = dat_out.rename({c_prev: c})
    dat_out_ds[c] = xr.concat([dat_out[c], xr_cov[c]], dim='time')
    
    dat_out_ds[c + '_despike'] = despike_ts_xr(dat_out_ds[c].chunk(smth_chunks),
                                               days_thresh=45,
                                               dat_thresh=0.3, 
                                               mask_outliers=True,
                                               z_thresh=5.0,
                                               iters=2,
                                               dims=['time'])
    dat_out_ds[c + '_smooth'] = smooth_xr(dat_out_ds[c + '_despike'], 
                                         dims=['time'],
                                         kwargs={'double': True, 'limit': 91})

    with rio.open(os.path.join(outDIR, 'hls_cover/' + prefix + '_hls_' + c + '_' + str(yr) + '.tif'), 
                  'w',
                  **out_profile) as dst:
        dst.write(dat_out_ds.sel(time=yr_mask)[c + '_smooth'].astype('float32'))
    
    c_prev = c

In [ ]:
ds_vars = xr.merge([xr_ndvi, xr_bm, xr_bm_se, xr_cov])

In [ ]:
print(ds_vars)

In [ ]:
print(dat_out)

In [ ]:
dat_out = dat_out.assign(NDVI=(['time', 'y', 'x'],
                               dat_out_nans))
dat_out = dat_out.assign(Biomass=(['time', 'y', 'x'],
                               dat_out_nans))
dat_out = dat_out.assign(Biomass_SE=(['time', 'y', 'x'],
                               dat_out_nans))
dat_out = dat_out.assign(BARE=(['time', 'y', 'x'],
                               dat_out_nans))
dat_out = dat_out.assign(SD=(['time', 'y', 'x'],
                               dat_out_nans))
dat_out = dat_out.assign(GREEN=(['time', 'y', 'x'],
                               dat_out_nans))
dat_out = dat_out.assign(LITT=(['time', 'y', 'x'],
                               dat_out_nans))

In [ ]:
dat_out_ds = xr.concat([dat_out['NDVI'], ds_vars['NDVI']], dim='time').to_dataset()
print(dat_out_ds)

In [ ]:
dat_out_ds['Biomass'] = xr.concat([dat_out['Biomass'], ds_vars['Biomass']], dim='time')

In [ ]:
dat_out_ds['Biomass_SE'] = xr.concat([dat_out['Biomass_SE'], ds_vars['Biomass_SE']], dim='time')

In [ ]:
#dat_out_ds['BARE'] = xr.concat([dat_out['BARE'], ds_vars['BARE']], dim='time')

In [ ]:
dat_out_ds['SD'] = xr.concat([dat_out['SD'], ds_vars['SD']], dim='time')

In [ ]:
dat_out_ds['GREEN'] = xr.concat([dat_out['GREEN'], ds_vars['GREEN']], dim='time')

In [ ]:
dat_out_ds['LITT'] = xr.concat([dat_out['LITT'], ds_vars['LITT']], dim='time')

In [ ]:
print(dat_out_ds)

In [ ]:
dat_out_ds = xr.concat([dat_out, ds_vars], dim='time')
#dat_out_ds = dat_out_ds.sortby('time')

In [ ]:
print(dat_out_ds.nbytes*(9.31*10e-10))

In [ ]:
print(ds_vars['BARE'])